## Task 1

In [2]:
#Python script to find out which visitor created the most revenue.

import sqlite3
connection = sqlite3.connect("transactions.db")
cursor = connection.cursor()
cursor.execute("SELECT visitor_id FROM Transactions ORDER BY revenue DESC LIMIT 1")
results = cursor.fetchone()[0]
print("The visitor who created the most revenue is:")
print(results)
cursor.close()
connection.close()

The visitor who created the most revenue is:
47447038460279


## Task 2

In [60]:
#Python script to find out on which day most revenue for users who ordered via a mobile phone was created

import sqlite3
connection = sqlite3.connect("transactions.db")
cursor = connection.cursor()
cursor.execute("SELECT DATE(datetime) FROM Transactions WHERE device_type in (SELECT id from Devices WHERE device_name = 'Mobile Phone') GROUP BY datetime ORDER BY sum(revenue) desc LIMIT 1")
results = cursor.fetchone()[0]
print("day most revenue for users who ordered via a mobile phone was created:")
print(results)
cursor.close()
connection.close()

day most revenue for users who ordered via a mobile phone was created:
2019-09-15


In [3]:
#printing the total revenue collected on the day 2019-09-15 by the users who ordered via mobile phone

import sqlite3
connection = sqlite3.connect("transactions.db")
cursor = connection.cursor()
cursor.execute("SELECT DATE(datetime),sum(revenue) FROM Transactions WHERE device_type in (SELECT id from Devices WHERE device_name = 'Mobile Phone') GROUP BY datetime ORDER BY sum(revenue) desc LIMIT 1 ")
results = cursor.fetchall()
for r in results:
    print(r)
cursor.close()
connection.close()

('2019-09-15', 20204.95143740077)


## Task 3

In [4]:
#Python script that combines the contents of Devices and Transactions and store it as a single flat file including the 
#column names

#output is stored in the file output.csv

import sqlite3 as db
import csv

# Running the query, the result is stored as 'data'
with db.connect('transactions.db') as conn:
    cur = conn.cursor()
    sql = "SELECT t.id,t.datetime,t.visitor_id,t.device_type,t.revenue,t.tax,d.device_name FROM Transactions as t JOIN Devices as d on t.device_type = d.id"
    cur.execute(sql)
    data = cur.fetchall()

# Create the csv file
with open('output.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    # Add the header/column names
    header = ['id', 'datetime', 'visitor_id', 'device_type','revenue','tax','device_name']
    writer.writerow(header)
    # Iterate over 'data'  and  write to the csv file
    for row in data:
        writer.writerow(row)

## Task 4

In [6]:
#Update the data stored in the database to have the created revenue in EUR.

import sqlite3
import xmltodict

conn = sqlite3.connect("transactions.db")
c = conn.cursor()

with open("eurofxref-hist-90d.xml") as fd:
    obj = xmltodict.parse(fd.read())

def extraction():
    for i in range(0,len(obj['gesmes:Envelope']['Cube']['Cube'])):
        date = obj['gesmes:Envelope']['Cube']['Cube'][i]['@time']
        rate = obj['gesmes:Envelope']['Cube']['Cube'][i]['Cube'][0]['@rate']
        frate = float(rate)
        update_task(conn, (frate, date))

def update_task(connection, task):
    """
    update Transactions
    :param conn:
    :param task:
    """
    sql = ''' UPDATE Transactions
              SET revenue = revenue * ? 
              WHERE DATE(datetime) = ?'''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

if __name__ == "__main__":
    extraction()


In [1]:
#printing the table value after updation. we can see that the revenue is converted to EUR
import sqlite3
connection = sqlite3.connect("transactions.db")
cursor = connection.cursor()
cursor.execute("SELECT * from Transactions LIMIT 10")
results = cursor.fetchall()
for r in results:
    print(r)
cursor.close()
connection.close()

(1, '2019-10-05 00:00:00', 1016870803637, 1, 173.3314596961336, 0.19)
(2, '2019-10-11 00:00:00', 48076008851343, 4, 320.36333994771235, 0.07)
(3, '2019-08-19 00:00:00', 72300317870324, 2, 567.8275631154788, 0.07)
(4, '2019-10-14 00:00:00', 84516511452821, 3, 655.1040690856495, 0.19)
(5, '2019-09-10 00:00:00', 44780848857001, 3, 744.4053360212473, 0.07)
(6, '2019-09-20 00:00:00', 54145933160851, 2, 65.88912097116538, 0.07)
(7, '2019-10-13 00:00:00', 68564288760762, 4, 512.0134481572289, 0.19)
(8, '2019-10-22 00:00:00', 74593067572606, 2, 189.36753885395905, 0.07)
(9, '2019-08-12 00:00:00', 74912374801029, 4, 965.7485939496092, 0.07)
(10, '2019-10-25 00:00:00', 67206600864292, 4, 32.94992557810085, 0.07)


## Task 5

Imagine you have to add support for other DBMS, how would you address this request?

From the provided Sqlite database, I generated csv files for transactions and Devices table(like i did in the task 3). Then I copied the csv files to postgresql using the below commands,

COPY public."Devices"(id, device_name) FROM 'C:\Users\Nila\Downloads\result\Devices.csv' DELIMITER ',' CSV HEADER;

COPY public."Transactions"(id,datetime,visitor_id,device_type,revenue,tax) FROM 'C:\Users\Nila\Downloads\result\Transactions.csv' DELIMITER ',' CSV HEADER;

Then, re-executed the task 2 with postgres database

In [ ]:
#creating csv file for transactions table

import sqlite3 as db
import csv

# Running the query, the result is stored as 'data'
with db.connect('transactions.db') as conn:
    cur = conn.cursor()
    sql = "SELECT * FROM Transactions"
    cur.execute(sql)
    data = cur.fetchall()

# Create the csv file
with open('transactions.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    # Add the header/column names
    header = ['id', 'datetime', 'visitor_id', 'device_type','revenue','tax']
    writer.writerow(header)
    # Iterate over 'data'  and  write to the csv file
    for row in data:
        writer.writerow(row)

In [ ]:
#creating csv file for devices table

import sqlite3 as db
import csv

# Running the query, the result is stored as 'data'
with db.connect('transactions.db') as conn:
    cur = conn.cursor()
    sql = "SELECT * FROM Devices"
    cur.execute(sql)
    data = cur.fetchall()

# Create the csv file
with open('Devices.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    # Add the header/column names
    header = ['id', 'device_name']
    writer.writerow(header)
    # Iterate over 'data'  and  write to the csv file
    for row in data:
        writer.writerow(row)

In [13]:
#Re-executing task2 with postgresql database.

import psycopg2
try:
    connection = psycopg2.connect(user="postgres",
                                  password="pass",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="transactions")
    cursor = connection.cursor()
    cursor.execute("""SELECT DATE(datetime),sum(revenue) FROM public."Transactions" WHERE device_type in (SELECT id from public."Devices" WHERE device_name = 'Mobile Phone') GROUP BY datetime ORDER BY sum(revenue) desc LIMIT 1 ;""")
    results = cursor.fetchone()[0]
    print("day most revenue for users who ordered via a mobile phone was created:")
    print(results)
except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

day most revenue for users who ordered via a mobile phone was created:
2019-09-15
PostgreSQL connection is closed
